In [2]:
import open3d as o3d
import numpy as np
import sys

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
sys.path.append('../sample')
from readdatas import *

In [6]:
path = '..\Point Cloud Data\Corner.ply'
pcd =  ReadDatas(path)

In [7]:
gpf = GpfGroundExtractor()
gpf.set_data(pcd)
gpf._process_data()

In [11]:
ground = pcd.ground
ground_o3d = gpf.np_to_o3d(pcd.ground)
ground_nor = np.asarray(ground_o3d.normals)

In [14]:
dps = DriPathSegment()
dps.set_data(pcd)
dps._process_data()

[Open3D INFO] Picked point #15094 (-14., -13., -1.6) to add in queue.
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 句柄无效。 
[Open3D INFO] Picked point #17045 (-5.8, -13., -1.6) to add in queue.
[Open3D INFO] Picked point #42547 (1.2, -11., -1.5) to add in queue.
[Open3D INFO] Picked point #57639 (4.5, -6.6, -1.5) to add in queue.
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 不支持请求的转换操作。 
[Open3D INFO] Picked point #68435 (5.2, -0.87, -1.5) to add in queue.
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 不支持请求的转换操作。 
[Open3D INFO] Picked point #70533 (5.0, 5.9, -1.5) to add in queue.


In [16]:
no_paves = pcd.no_paves

In [7]:
ground_tree = o3d.geometry.KDTreeFlann(ground_o3d)


In [8]:
n = len(ground)
feature_vector = np.zeros((n,2))


In [10]:
for i in range(n):
    [num, idx, _] = ground_tree.search_radius_vector_3d(ground_o3d.points[i], 0.2)
    point_neighbour = ground[idx,:]
    point_neighbour_max = point_neighbour[:,2].max(axis=0)
    point_neighbour_min = point_neighbour[:,2].min(axis=0)
    point_neighbour_h = point_neighbour_max - point_neighbour_min
    point_neighbour_var = np.var(point_neighbour[:,2],axis=0)
    feature_vector[i,:] = [point_neighbour_h,point_neighbour_var]



In [11]:
feature_vector = np.concatenate((feature_vector,ground_nor),axis=1)


In [12]:
driving_track = datas.driving_path_generation(point,ground_o3d)

In [13]:
driving_track_o3d = datas.np_to_o3d(driving_track)

In [14]:
driving_track_o3d = datas.find_nearest_point(ground_o3d, driving_track_o3d)


In [15]:
driving_track_o3d = ground_o3d.select_by_index(driving_track_o3d)


In [16]:
o3d.visualization.draw_geometries([driving_track_o3d])


In [17]:
point_neighbours = datas.find_nearest_point(driving_track_o3d,ground_o3d)

In [18]:
def driving_path_extraction(ground,driving_track,point_neighbours):
        ground_np = np.asarray(ground.points)
        ground_nom = ground.estimate_normals()
        ground_nom = np.asarray(ground.normals)
        theta_threshold = 30
        cosine_threshold = np.cos(np.deg2rad(theta_threshold))

        no_paves = np.array([0])
        index_all = np.array(range(len(ground_np)))
        for i in range(len(driving_track)):
            slim = ground_np[point_neighbours[:,0] == i]
            slim_index = index_all[point_neighbours[:,0] == i]
            if len(slim) > 0:
                slim_distance = np.sqrt(np.sum(np.power(slim - driving_track[i,:],2),axis = 1))
                p = slim_distance.argmin()
                slim_nom = slim - driving_track[i,:]
                slim_distance[slim_nom[:,0]/slim_nom[:,1] < 0] = slim_distance[slim_nom[:,0]/slim_nom[:,1] < 0] * -1
                slim = slim[slim_distance.argsort()]
                slim_index = slim_index[slim_distance.argsort()]
                slim_nor = ground_nom[point_neighbours[:,0] == i]
                p = slim_nor[p,:]
                slim_nor = slim_nor[slim_distance.argsort()]
                slim_distance.sort()
                a_jiao = np.fabs(np.dot(slim_nor, p))
                # b = np.diff(slim[:,2])
                # b =np.append(b,1)
                a = a_jiao > cosine_threshold
                c = a_jiao <= cosine_threshold
                d = slim[a]
                no_paves = np.concatenate((no_paves,slim_index[c]),axis=0)
                
        no_paves = np.delete(no_paves,0,0)

        return no_paves

In [19]:
driving_track = np.asarray(driving_track_o3d.points)

In [20]:
no_paves = driving_path_extraction(ground_o3d,driving_track,point_neighbours)

C:\Users\master\AppData\Local\Temp\ipykernel_12104\845922707.py:17: RuntimeWarning: invalid value encountered in divide
  slim_distance[slim_nom[:,0]/slim_nom[:,1] < 0] = slim_distance[slim_nom[:,0]/slim_nom[:,1] < 0] * -1
C:\Users\master\AppData\Local\Temp\ipykernel_12104\845922707.py:17: RuntimeWarning: divide by zero encountered in divide
  slim_distance[slim_nom[:,0]/slim_nom[:,1] < 0] = slim_distance[slim_nom[:,0]/slim_nom[:,1] < 0] * -1


In [21]:
y = np.full(len(ground),False,dtype=bool)

In [22]:
y[no_paves] = True

In [23]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

In [24]:
# MLP神经网络
clf = MLPClassifier(hidden_layer_sizes=(5, 2), random_state=0)
clf.fit(feature_vector, y)

MLPClassifier(hidden_layer_sizes=(5, 2), random_state=0)

In [59]:


# 创建一个SVM模型，使用RBF核函数
clf = svm.SVC(kernel='rbf', C=1, gamma=0.1)

# 训练模型
clf.fit(feature_vector, y)


SVC(C=1, gamma=0.1)

In [29]:

# 使用模型进行预测
y_pred = clf.predict(feature_vector)


In [24]:
import joblib
# joblib.dump(clf,'MLP_model.pkl')


In [25]:
clf_loaded = joblib.load('model.pkl')

In [26]:
y_pred = clf_loaded.predict(feature_vector[no_paves])


In [27]:
y_pred_all = clf_loaded.predict(feature_vector)

In [28]:
accuracy = accuracy_score(y[no_paves], y_pred)

# 输出准确率
print("Accuracy: %0.2f" % accuracy)

Accuracy: 0.81


In [29]:
accuracy_all = accuracy_score(y, y_pred_all)

# 输出准确率
print("Accuracy: %0.2f" % accuracy_all)

Accuracy: 0.98


In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.utils import formataddr


In [ ]:

# 邮件发送方的信息
smtp_server = "smtp.qq.com"  # QQ邮箱的SMTP服务器地址
smtp_port = 465  # QQ邮箱的SMTP服务器端口号
smtp_username = "1874032283@qq.com"  # 邮件发送方的QQ邮箱地址
smtp_password = "pjfgeajjemwfiacc"  # 邮件发送方的QQ邮箱授权码，不是QQ登录密码
from_addr = "1874032283@qq.com"  # 邮件发送方的QQ邮箱地址


In [ ]:

# 邮件接收方的信息
to_addr = "1874032283@qq.com"  # 邮件接收方的邮箱地址

# 邮件内容
msg = MIMEText("Accuracy: %0.2f" % accuracy,"Accuracy_all: %0.2f" % accuracy_all)  # 邮件内容
msg['From'] = formataddr(("python程序", from_addr))  # 发件人昵称，可自定义
msg['To'] = to_addr
msg['Subject'] = "实验结果"

# 发送邮件
server = smtplib.SMTP_SSL(smtp_server, smtp_port)  # 使用SMTP_SSL类连接QQ邮箱的SMTP服务器
server.login(smtp_username, smtp_password)
server.sendmail(from_addr, [to_addr], msg.as_string())
server.quit()


(221, b'Bye.')